In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax', bias_regularizer=tf.keras.regularizers.l2(0.01))
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [ ]:
history = model.fit(train_data,
                    epochs=30,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 15:51:19.011088: {'loss': 1.7453446639299393, 'accuracy': 0.52, 'val_loss': 1.0222094408266105, 'val_accuracy': 0.6519}
Epoch 1 ends at 15:55:27.302223: {'loss': 0.7599972515106201, 'accuracy': 0.7406, 'val_loss': 0.7763329205239654, 'val_accuracy': 0.7472}
Epoch 2 ends at 15:59:45.385301: {'loss': 0.4555173779428005, 'accuracy': 0.84178126, 'val_loss': 0.7637511476589616, 'val_accuracy': 0.7573}
Epoch 3 ends at 16:04:03.914538: {'loss': 0.281345135255158, 'accuracy': 0.9011437, 'val_loss': 0.8794279343383328, 'val_accuracy': 0.7698}
Epoch 4 ends at 16:08:24.480960: {'loss': 0.18339385634586214, 'accuracy': 0.9359375, 'val_loss': 1.0393373681481477, 'val_accuracy': 0.7531}
Epoch 5 ends at 16:12:52.908057: {'loss': 0.14016855713557452, 'accuracy': 0.9509875, 'val_loss': 0.9831283952400183, 'val_accuracy': 0.7843}
Epoch 6 ends at 16:17:17.611680: {'loss': 0.11495254803262651, 'accuracy': 0.96045625, 'val_loss': 0.8752877064950907, 'val_accuracy': 0.8014}
Epoch 7 ends at 1

In [ ]:
model.evaluate(test_data)

In [ ]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))